In [22]:
import keras 
from keras import layers
from keras import optimizers
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
#import cv2 as cv

In [25]:
## load the alzheimers dataset
def load_data(test_split = 0.3):
    
    ## (1) Data preparation
    df=pd.read_csv('AlzData.csv', sep = ',')
    print(df)
    
    # rm ids, rm targets
    transposedDataFrameVals = df.values.T[1:-1]
    #walk thru features
    for i, row in enumerate(transposedDataFrameVals):
        #normalize data [0,1]
        transposedDataFrameVals[i] = row.astype(float) / max(row.astype(float))
    
    #assign normalized data
    X = transposedDataFrameVals.T
    #assign targets
    Y = df.values.T[-1]

    # data split of 70 training and 30 test
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=test_split)
        
    return x_train, y_train, x_test, y_test

x_train, y_train, x_test, y_test = load_data()

print('Data shape:', 'x_train:', x_train.shape, 'x_test:', x_test.shape)
print('Data shape:', 'y_train:', y_train.shape, 'y_test:', y_test.shape)


Data shape: x_train: (121, 450) x_test: (53, 450)
Data shape: y_train: (121,) y_test: (53,)


In [55]:
# expand dims for channels?

desiredLevels = 2

def CreateConvLayer(convLayers, model, filters, kernel_size, activation = None, padding = "valid"):
    conv = layers.Conv2D(
        filters= filters, #num of filters for conv
        kernel_size = kernel_size, 
        padding = padding,
        activation = activation,
        #input_shape = (28, 28, 1) #only 28 params 
    )
    
    convLayers.append(conv)
    model.add(conv)
    
def CreateMaxPoolLayer(poolLayers, model, pool_size, strides):
    pool = layers.MaxPooling2D(
        pool_size = pool_size,
        strides = strides,
    )
    
    poolLayers.append(pool)
    model.add(pool)
    
def CreateConvBlock(
    model, convLayers, poolLayers, normLayers, activationLayers,
    filters, kernel_size, activation,
    pool_size, strides
):
    CreateConvLayer(convLayers, model, filters, kernel_size)
    #CreateMaxPoolLayer(poolLayers, model, pool_size, strides)
    
    dropout = layers.Dropout(0.2)
    model.add(dropout)
    
    norm = layers.BatchNormalization()
    model.add(norm)
    normLayers.append(norm)

    activ = layers.Activation(activation)
    model.add(activ)
    activationLayers.append(activ)
    
# Create a fully conv NN
model_fcn = keras.Sequential()

convLayers = []
poolLayers = []
normLayers = []
activationLayers = []

# Input layer
input = layers.Input(shape=(None, None, 1))
model_fcn.add(input)

CreateConvBlock(
    model_fcn, convLayers, poolLayers, normLayers, activationLayers,
    filters=8, kernel_size=3, activation='relu',
    pool_size=2, strides=2
)

CreateConvBlock(
    model_fcn, convLayers, poolLayers, normLayers, activationLayers,
    filters=16, kernel_size=3, activation='softmax',
    pool_size=2, strides=2
)

globMaxPool = layers.GlobalMaxPooling2D()
model_fcn.add(globMaxPool)

model_fcn(inputs=x_train, outputs=y_train)

#need normalization and output layers

#specify optimization
model_fcn.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model_fcn.summary()



Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, None, None, 8)     80        
                                                                 
 dropout_2 (Dropout)         (None, None, None, 8)     0         
                                                                 
 batch_normalization_2 (Batc  (None, None, None, 8)    32        
 hNormalization)                                                 
                                                                 
 activation_2 (Activation)   (None, None, None, 8)     0         
                                                                 
 conv2d_3 (Conv2D)           (None, None, None, 16)    1168      
                                                                 
 dropout_3 (Dropout)         (None, None, None, 16)    0         
                                                      

In [ ]:
#train model
history = model_fcn.fit(
    x_train, #should be 2d inputs
    y_train, #already onehot
    batch_size=64,
    epochs=10,
    verbose=1,
    validation_split=0.2,
)

In [ ]:
#test accuracy
y_test_pred = model_fcn.predict(x_test)
y_test_pred = np.argmax(y_test_pred, axis=1) #still need to do bc pred is prob?
test_err = np.sum(y_test == y_test_pred) / y_test.shape[0]

print(f'test accuracy: {test_err * 100}')

In [ ]:
#history of training and validation accuracu
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['training acc', 'validation acc'])

In [ ]:
#get access to filters/kernels
weights = convLayers[0].get_weights()[0][:, :, 0, :] #arr of a buncha nested lists, 0 bc only have 1 channel (28, 28, 1, 8)

#can get access to other params this way 

for i in range(1, 8):
    plt.subplot(2, 4, i)
    plt.imshow(weights[:,:,i], cmap="gray")
plt.show()

#how get access to feature maps?